In [ ]:
"""
{
  "query": [
    {
      "code": "Region",
      "selection": {
        "filter": "vs:KommunerNyeste",
        "values": [
          "*"
        ]
      }
    },
    {
      "code": "Alder",
      "selection": {
        "filter": "vs:AlleAldre00B",
        "values": []
      }
    },
    {
      "code": "Tid",
      "selection": {
        "filter": "item",
        "values": [
          "2018"
        ]
      }
    }
  ],
  "response": {
    "format": "csv"
  }
}
"""

In [1]:
import requests

## Earnings

In [80]:
URL = "https://data.ssb.no/api/v0/no/table/11418/"

query = {
  "query": [
    {
      "code": "MaaleMetode",
      "selection": {
        "filter": "item",
        "values": [
          "02",
          "01",
          "10"
        ]
      }
    },
    {
      "code": "Yrke",
      "selection": {
        "filter": "all",
        "values": [
         "*"
        ]
      }
    },
    {
      "code": "Sektor",
      "selection": {
        "filter": "item",
        "values": [
          "ALLE"
        ]
      }
    },
    {
      "code": "Kjonn",
      "selection": {
        "filter": "item",
        "values": [
          "0"
        ]
      }
    },
    {
      "code": "AvtaltVanlig",
      "selection": {
        "filter": "item",
        "values": [
          "0"
        ]
      }
    },
    {
      "code": "ContentsCode",
      "selection": {
        "filter": "item",
        "values": [
          "Manedslonn"
        ]
      }
    },
    {
      "code": "Tid",
      "selection": {
        "filter": "item",
        "values": [
          "2019"
        ]
      }
    }
  ],
  "response": {
    "format": "csv"
  }
}

In [81]:
response = requests.post(URL, json=query)

In [82]:
response.status_code

200

In [197]:
# Just illustrating lines in file
response.text.split('\r\n')[:2]

['"statistikkmål","yrke","sektor","kjønn","avtalt/vanlig arbeidstid per uke","Månedslønn (kr) 2019"',
 '"Gjennomsnitt","0-9 Alle yrker","Sum alle sektorer","Begge kjønn","I alt",47290']

In [ ]:
len(response.text)

In [99]:
def make_df(csvstring, names=None, dtypes=None, header=None, skiprows=None):
    import pandas as pd
    from io import StringIO
    df = pd.read_csv(StringIO(csvstring), names=names, dtype=dtypes, header=header, skiprows=skiprows)
    return(df)

In [226]:
df = make_df(response.text, names=['measure', 'ISCO_text', 'sektor', 'kjonn', 'arbtid', 'lonn'], skiprows=1)

In [227]:
df = (df[['measure', 'ISCO_text', 'lonn']]
      .query('lonn!=":"')
      .loc[df["ISCO_text"].apply(lambda x: True if re.match('[0-9]{4}', x[:4]) else False )]
      .assign(lonn = pd.to_numeric(df["lonn"], errors="coerce") )
      .pivot(index='ISCO_text', columns='measure', values='lonn')
      .reset_index()
      .assign(ISCO = lambda x: x["ISCO_text"].str[:4])
      )

In [228]:
df.head()

measure,ISCO_text,Antall arbeidsforhold med lønn,Gjennomsnitt,Median,ISCO
0,0110 Offiserer fra fenrik og høyere grad,9114.0,63990.0,59200.0,0110
1,0210 Befal med sersjant grad,1193.0,46810.0,43240.0,0210
2,1111 Politikere,1830.0,68410.0,69390.0,1111
3,1112 Toppledere i offentlig administrasjon,1028.0,97940.0,93750.0,1112
4,1114 Toppledere i interesseorganisasjoner,2344.0,62890.0,58330.0,1114


In [229]:
df.columns = ['ISCO_text', 'antall', 'snitt', 'median', 'ISCO']

In [230]:
df.dtypes

ISCO_text     object
antall       float64
snitt        float64
median       float64
ISCO          object
dtype: object

In [231]:
df.head()

,ISCO_text,antall,snitt,median,ISCO
0,0110 Offiserer fra fenrik og høyere grad,9114.0,63990.0,59200.0,0110
1,0210 Befal med sersjant grad,1193.0,46810.0,43240.0,0210
2,1111 Politikere,1830.0,68410.0,69390.0,1111
3,1112 Toppledere i offentlig administrasjon,1028.0,97940.0,93750.0,1112
4,1114 Toppledere i interesseorganisasjoner,2344.0,62890.0,58330.0,1114


In [232]:
df.to_feather("workdata/earnings.feather")

## Sysselsetting kommune

In [ ]:
URL = "https://data.ssb.no/api/v0/no/table/11619"

query = {
  "query": [
    {
      "code": "Region",
      "selection": {
        "filter": "all",
        "values": [
          "*"
        ]
      }
    },
    {
      "code": "Kjonn",
      "selection": {
        "filter": "item",
        "values": [
          "0"
        ]
      }
    },
    {
      "code": "Alder",
      "selection": {
        "filter": "item",
        "values": [
          "15-74"
        ]
      }
    },
    {
      "code": "Yrke",
      "selection": {
        "filter": "item",
        "values": [
          "1",
          "2",
          "3_01-03",
          "4",
          "5",
          "6",
          "7",
          "8",
          "9"
        ]
      }
    },
    {
      "code": "Tid",
      "selection": {
        "filter": "item",
        "values": [
          "2018"
        ]
      }
    }
  ],
  "response": {
    "format": "csv"
  }
}

In [ ]:
response = requests.post(URL, json=query)

In [ ]:
response.status_code

In [ ]:
response.text

In [78]:
df.head()

,statistikkmål,yrke,sektor,kjønn,avtalt/vanlig arbeidstid per uke,Månedslønn (kr) 2019
0,Gjennomsnitt,0-9 Alle yrker,Sum alle sektorer,Begge kjønn,I alt,47290
1,Gjennomsnitt,00 Uoppgitt / yrker som ikke kan identifiseres,Sum alle sektorer,Begge kjønn,I alt,:
2,Gjennomsnitt,000 Uoppgitt / yrker som ikke kan identifiseres,Sum alle sektorer,Begge kjønn,I alt,:
3,Gjennomsnitt,0000 Uoppgitt / yrker som ikke kan identifiseres,Sum alle sektorer,Begge kjønn,I alt,:
4,Gjennomsnitt,01 Offiserer fra fenrik og høyere grad,Sum alle sektorer,Begge kjønn,I alt,63990


In [79]:
df.columns

Index(['statistikkmål', 'yrke', 'sektor', 'kjønn',
       'avtalt/vanlig arbeidstid per uke', 'Månedslønn (kr) 2019'],
      dtype='object')

## Area

In [ ]:
URL = "https://data.ssb.no/api/v0/no/table/11342"

query = {
  "query": [
    {
      "code": "Region",
      "selection": {
        "filter": "all",
        "values": [
          "*"
        ]
      }
    },
    {
      "code": "ContentsCode",
      "selection": {
        "filter": "item",
        "values": [
          "Folkemengde",
          "ArealKm2",
          "LandArealKm2",
          "FolkeLandArealKm2"
        ]
      }
    },
    {
      "code": "Tid",
      "selection": {
        "filter": "item",
        "values": [
          "2019"
        ]
      }
    }
  ],
  "response": {
    "format": "csv"
  }
}


In [ ]:
response = requests.post(URL, json=query)
response.status_code

In [ ]:
response.text

## Employed

In [2]:
URL = 'https://data.ssb.no/api/v0/no/table/12542/'

query = {
  "query": [
    {
      "code": "Yrke",
      "selection": {
        "filter": "all",
        "values": [
          "*"
        ]
      }
    },
    {
      "code": "Kjonn",
      "selection": {
        "filter": "item",
        "values": [
          "0"
        ]
      }
    },
    {
      "code": "ArbeidsTidRen",
      "selection": {
        "filter": "vs:ArbTidS11",
        "values": [
          "P000-100"
        ]
      }
    },
    {
      "code": "Tid",
      "selection": {
        "filter": "item",
        "values": [
          "2019"
        ]
      }
    }
  ],
  "response": {
    "format": "csv"
  }
}

In [3]:
response = requests.post(URL, json=query)
response.status_code

200

In [4]:
len(response.text)

35776

In [15]:
import pandas as pd
from io import StringIO
syss = pd.read_csv(StringIO(response.text))

In [18]:
syss.head()

,yrke,kjønn,arbeidstid,Lønnstakere 2019
0,0-9 Alle yrker,Begge kjønn,I alt,2562491
1,00 Uoppgitt / yrker som ikke kan identifiseres,Begge kjønn,I alt,9057
2,000 Uoppgitt / yrker som ikke kan identifiseres,Begge kjønn,I alt,9057
3,0000 Uoppgitt / yrker som ikke kan identifiseres,Begge kjønn,I alt,9057
4,01 Offiserer fra fenrik og høyere grad,Begge kjønn,I alt,9015


In [19]:
syss["ISCO"] = syss["yrke"].str[:4]

In [47]:
import re

syss_isco4 = syss.loc[syss["yrke"].apply(lambda x: True if re.match('[0-9]{4}', x[:4]) else False )]

In [48]:
len(syss)

582

In [49]:
len(syss_isco4)

407

In [52]:
syss_isco4.to_csv('master_data/syss_check.csv', index=False)

In [63]:
syss_isco4.reset_index()[["ISCO", "Lønnstakere 2019"]].to_csv('master_data/syss_check.csv')